In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 


In [ ]:
# How should I select the genomes for model training?
genome_ids = ['GCF_000005845.2'] # E. coli K-12
genome_ids += ['GCF_000009045.1'] # B. subtilis
genome_ids += ['GCF_000006765.1'] # P. aeruginosa
genome_ids += ['GCF_000195955.2'] # M. tuberculosis

names = dict()
names['GCF_000005845.2'] = 'E. coli K-12'
names['GCF_000009045.1'] = 'B. subtilis 168'
names['GCF_000006765.1'] = 'P. aeruginosa'
names['GCF_000195955.2'] = 'M. tuberculosis'

In [ ]:
# What statistics do I care about?
# (1) Total number of genes in reference. 
# (2) Total number of predicted genes. 
# (3) Total number of matched coding sequences (not including pseudogenes).
# (4) Number of exact matches. 
# (5) Number of genes with boundary errors (characterize the nature of the boundary errors later on).
# (2) The false discovery rate. 
# (3) The number of missed genes. 